In [4]:
import pymysql.cursors
import jsonify
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [5]:
conn = pymysql.connect(
    host="softwaredb.ce0otalnccc9.eu-west-1.rds.amazonaws.com",user="soft",password="password",database="dublinbikes",
    charset="utf8mb4",
    cursorclass=pymysql.cursors.DictCursor
)


In [7]:
availability_query = "SELECT number, time, available_bikes, available_bike_stands FROM availability"
#weather_query = "SELECT number, time, temperature, wind_speed, wind_direction, weather_code FROM weather"
weather_query = "SELECT * FROM weather"
# Load the query results into pandas dataframes
availability_df = pd.read_sql_query(availability_query, conn)
weather_df = pd.read_sql_query(weather_query, conn)

# Merge the dataframes on "time" and "number" columns



# Close the database connection
conn.close()

C:\Users\tfiro\AppData\Local\Temp\ipykernel_26120\2178617842.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  availability_df = pd.read_sql_query(availability_query, conn)


DatabaseError: Execution failed on sql: SELECT number, time, available_bikes, available_bike_stands FROM availability
(0, '')
unable to rollback

In [9]:
availability_df['time'] = availability_df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
weather_df['time'] = weather_df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
merged_df = pd.merge(availability_df, weather_df, on=["number", "time"])

In [10]:
merged_df.to_csv('merged_data.csv', index=False)

In [28]:
merged_df.head()

,number,time,available_bikes,available_bike_stands,temperature,wind_speed,wind_direction,weather_code,precipitation_sum,rain_sum,precipitation_probability,hour
0,1,2023-03-07 19:25:03,12,19,3,8,63,0,0.0,0.0,6,19
1,1,2023-03-07 19:30:02,12,19,3,8,63,0,0.0,0.0,6,19
2,1,2023-03-07 19:35:03,12,19,3,8,63,0,0.0,0.0,6,19
3,1,2023-03-07 19:40:02,12,19,3,8,63,0,0.0,0.0,6,19
4,1,2023-03-07 19:45:02,13,18,3,8,63,0,0.0,0.0,6,19


In [27]:
merged_df.isnull().sum()

number                       0
time                         0
available_bikes              0
available_bike_stands        0
temperature                  0
wind_speed                   0
wind_direction               0
weather_code                 0
precipitation_sum            0
rain_sum                     0
precipitation_probability    0
dtype: int64

In [29]:
merged_df.describe().T

,count,mean,std,min,25%,50%,75%,max
number,976365.0,58.950337,36.573246,1.0,29.0,58.0,89.0,507.0
available_bikes,976365.0,11.608905,9.494840,0.0,4.0,10.0,17.0,40.0
available_bike_stands,976365.0,19.964370,10.934173,0.0,12.0,20.0,29.0,40.0
temperature,976365.0,8.988125,3.363725,1.0,7.0,10.0,11.0,17.0
wind_speed,976365.0,16.884550,8.229185,1.0,11.0,16.0,22.0,44.0
wind_direction,976365.0,197.415392,77.332009,3.0,145.0,213.0,259.0,360.0
weather_code,976365.0,18.028062,23.979019,0.0,2.0,3.0,51.0,73.0
precipitation_sum,976365.0,3.860875,5.504092,0.0,0.4,2.0,4.5,30.2
rain_sum,976365.0,2.645874,3.932484,0.0,0.2,0.7,3.3,20.2
precipitation_probability,976365.0,77.984557,29.218425,0.0,61.0,94.0,100.0,100.0


In [46]:
print(merged_df.dtypes)


number                             category
time                         datetime64[ns]
available_bikes                       int64
available_bike_stands                 int64
temperature                           int64
wind_speed                            int64
wind_direction                        int64
weather_code                       category
precipitation_sum                   float64
rain_sum                            float64
precipitation_probability             int64
hour                               category
minute                             category
month                              category
day                                category
dtype: object


In [29]:
merged_df['time'] = merged_df['time'].astype('datetime64')
merged_df['number'] = merged_df['number'].astype('category')
merged_df['weather_code'] = merged_df['weather_code'].astype('category')

C:\Users\tfiro\AppData\Local\Temp\ipykernel_26120\2672415241.py:1: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  merged_df['time'] = merged_df['time'].astype('datetime64')


In [37]:
merged_df['hour'] = merged_df['time'].dt.hour
merged_df['minute'] = merged_df['time'].dt.minute
merged_df['month'] = merged_df['time'].dt.month
merged_df['day'] = merged_df['time'].dt.day


In [45]:
merged_df['hour'] = merged_df['time'].astype('category')
merged_df['minute'] = merged_df['time'].astype('category')
merged_df['month'] = merged_df['time'].astype('category')
merged_df['day'] = merged_df['time'].astype('category')

In [40]:
station_dfs = {}
for station in merged_df['number'].unique():
    station_dfs[station] = merged_df[merged_df['number'] == station]


In [ ]:
from sklearn.ensemble import RandomForestRegressor
for station, station_df in station_dfs.items():
    X = station_df[['hour', 'day','minute','month','temperature', 'wind_speed', 'wind_direction', 'weather_code']]
    y = station_df['available_bikes']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)

    model.fit(X_train, y_train)
    print(f"Station {station} R-squared: {model.score(X_test, y_test)}")
    station_num = 1
    time_str = '2023-03-07 19:35:03'
    weather_code = 801
    temperature = 10
    wind_speed = 6
    wind_direction = 240
    time = pd.to_datetime(time_str)
    hour = time.hour
    day=time.day
    minute=time.minute
    month=time.month
    prediction_input = [[hour,day,minute,month, temperature, wind_speed, wind_direction, weather_code]]
    prediction = model.predict(prediction_input)
    
    print(f"Predicted number of available bikes for Station {station} at {time_str}: {prediction[0]:.2f}")


Station 1 R-squared: 0.9885572626983842
Predicted number of available bikes for Station 1 at 2023-03-07 19:35:03: 10.09


C:\Users\tfiro\anaconda3\envs\data_analytics\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Station 2 R-squared: 0.9656947068603603
Predicted number of available bikes for Station 2 at 2023-03-07 19:35:03: 6.76


C:\Users\tfiro\anaconda3\envs\data_analytics\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Station 3 R-squared: 0.9806680733824458
Predicted number of available bikes for Station 3 at 2023-03-07 19:35:03: 16.48


...

C:\Users\tfiro\anaconda3\envs\data_analytics\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Station 117 R-squared: 0.9969065388949339
Predicted number of available bikes for Station 117 at 2023-03-07 19:35:03: 5.70


C:\Users\tfiro\anaconda3\envs\data_analytics\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Station 507 R-squared: 1.0
Predicted number of available bikes for Station 507 at 2023-03-07 19:35:03: 1.00


C:\Users\tfiro\anaconda3\envs\data_analytics\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [43]:
import pickle
from sklearn.ensemble import RandomForestRegressor

for station, station_df in station_dfs.items():
    X = station_df[['hour', 'day','minute','month','temperature', 'wind_speed', 'wind_direction', 'weather_code']]
    y = station_df['available_bikes']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)

    model.fit(X_train, y_train)
    print(f"Station {station} R-squared: {model.score(X_test, y_test)}")
    
    #Save the model to a pickle file
    filename = f"model_dep{station}.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(model, file, pickle.HIGHEST_PROTOCOL)


Station 1 R-squared: 0.9885572626983842
Station 2 R-squared: 0.9656947068603603
Station 3 R-squared: 0.9806680733824458
Station 4 R-squared: 0.9892477221003506
Station 5 R-squared: 0.987207239004247
Station 6 R-squared: 0.9910299938001904
Station 7 R-squared: 0.9877670110437442
Station 8 R-squared: 0.9802049022405906
Station 9 R-squared: 0.9720537872703916
Station 10 R-squared: 0.9705409899949446
Station 11 R-squared: 0.995218162354485
Station 12 R-squared: 0.9851238314230923
Station 13 R-squared: 0.994618784266364
Station 14 R-squared: 0.9736102373177122
Station 15 R-squared: 0.9592645201140807
Station 16 R-squared: 0.9776311273965224
Station 17 R-squared: 0.9889064390473342
Station 18 R-squared: 0.9852430276976931
Station 19 R-squared: 0.9901850380719291
Station 20 R-squared: 0.994070236133207
Station 21 R-squared: 0.9831702824885127
Station 22 R-squared: 0.9846415571586682
Station 23 R-squared: 0.9843041265312743
Station 24 R-squared: 0.9641325959468358
Station 25 R-squared: 0.99513

In [44]:
import pickle
from sklearn.ensemble import RandomForestRegressor

for station, station_df in station_dfs.items():
    X = station_df[['hour', 'day','minute','month','temperature', 'wind_speed', 'wind_direction', 'weather_code']]
    y = station_df['available_bike_stands']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)

    model.fit(X_train, y_train)
    print(f"Station {station} R-squared: {model.score(X_test, y_test)}")
    
    #Save the model to a pickle file
    filename = f"model_arr{station}.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(model, file, pickle.HIGHEST_PROTOCOL)


Station 1 R-squared: 0.9883671791530877
Station 2 R-squared: 0.966144045680856
Station 3 R-squared: 0.9806294228142073
Station 4 R-squared: 0.9890948586540731
Station 5 R-squared: 0.9870281086616738
Station 6 R-squared: 0.9907493064762244
Station 7 R-squared: 0.9876425174906951
Station 8 R-squared: 0.9802282805607118
Station 9 R-squared: 0.9723913941447229
Station 10 R-squared: 0.9700441686692782
Station 11 R-squared: 0.9952032187962234
Station 12 R-squared: 0.98518101677666
Station 13 R-squared: 0.9948387599078815
Station 14 R-squared: 0.9738449097065885
Station 15 R-squared: 0.9584663017866462
Station 16 R-squared: 0.9780724838948803
Station 17 R-squared: 0.9894295290716049
Station 18 R-squared: 0.9852471910879688
Station 19 R-squared: 0.9905007306960811
Station 20 R-squared: 0.9940980364027817
Station 21 R-squared: 0.9830084581301374
Station 22 R-squared: 0.9843441385853734
Station 23 R-squared: 0.9736510503311016
Station 24 R-squared: 0.9641852731137218
Station 25 R-squared: 0.9954